<a href="https://colab.research.google.com/github/keduog/LLM/blob/main/institution_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas openpyxl requests PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [6]:
# STEP 0: Upload the Excel file using Colab UI
from google.colab import files
uploaded = files.upload()

import pandas as pd
import requests
from io import BytesIO
from PyPDF2 import PdfReader
import re
import time
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

# STEP 1: Load Excel file (first uploaded)
excel_filename = list(uploaded.keys())[0]
df = pd.read_excel(excel_filename)

if 'url' not in df.columns:
    raise ValueError("Excel file must contain a column named 'url'")

urls = df['url'].dropna().tolist()

# STEP 2: Utility functions
def get_pdf_url(acl_url):
    if acl_url.endswith('/'):
        acl_url = acl_url[:-1]
    paper_id = acl_url.split('/')[-1]
    return f"https://aclanthology.org/{paper_id}.pdf"

def extract_affiliations(text):
    lines = text.split("\n")
    affiliations = set()
    for line in lines:
        if len(line.strip()) < 5 or '@' in line:
            continue
        if any(keyword in line.lower() for keyword in [
            'university', 'institute', 'college', 'school', 'laboratory', 'lab',
            'dept.', 'department', 'center', 'centre', 'academy'
        ]):
            cleaned = re.sub(r'\b[\w.-]+?@\w+?\.\w+?\b', '', line)
            cleaned = re.sub(r'\s+', ' ', cleaned.strip())
            cleaned = ILLEGAL_CHARACTERS_RE.sub('', cleaned)  # remove illegal Excel characters
            if 5 < len(cleaned) < 200:
                affiliations.add(cleaned)
    return sorted(affiliations)

# STEP 3: Process URLs
output = []
for idx, url in enumerate(urls, 1):
    print(f"[{idx}/{len(urls)}] Processing: {url}")
    try:
        pdf_url = get_pdf_url(url)
        response = requests.get(pdf_url, timeout=30)
        response.raise_for_status()
        reader = PdfReader(BytesIO(response.content))
        first_page = reader.pages[0].extract_text()
        institutions = extract_affiliations(first_page)
        cleaned_institutions = [ILLEGAL_CHARACTERS_RE.sub('', inst) for inst in institutions]
        output.append({
            "URL": url,
            "Institutions": "; ".join(cleaned_institutions) if cleaned_institutions else "Not found"
        })
        time.sleep(1)
    except Exception as e:
        output.append({
            "URL": url,
            "Institutions": f"ERROR: {str(e)}"
        })

# STEP 4: Save and download result
result_df = pd.DataFrame(output)
output_file = "affiliations_extracted.xlsx"
result_df.to_excel(output_file, index=False)

print("\n✅ Done! Download your file below:")
files.download(output_file)


Saving institution.xlsx to institution (1).xlsx
[1/1902] Processing: https://aclanthology.org/2025.wacl-1.1/
[2/1902] Processing: https://aclanthology.org/2025.wacl-1.9/
[3/1902] Processing: https://aclanthology.org/2025.wacl-1.10/
[4/1902] Processing: https://aclanthology.org/2025.resourceful-1.34/
[5/1902] Processing: https://aclanthology.org/2025.nlp4dh-1.26/
[6/1902] Processing: https://aclanthology.org/2025.naacl-long.92/
[7/1902] Processing: https://aclanthology.org/2025.naacl-long.139/
[8/1902] Processing: https://aclanthology.org/2025.naacl-long.338/
[9/1902] Processing: https://aclanthology.org/2025.naacl-long.426/
[10/1902] Processing: https://aclanthology.org/2025.naacl-long.602/
[11/1902] Processing: https://aclanthology.org/2025.naacl-short.2/
[12/1902] Processing: https://aclanthology.org/2025.naacl-short.35/
[13/1902] Processing: https://aclanthology.org/2025.loresmt-1.4/
[14/1902] Processing: https://aclanthology.org/2025.loresmt-1.6/
[15/1902] Processing: https://aclan

[334/1902] Processing: https://aclanthology.org/2021.mtsummit-research.6/
[335/1902] Processing: https://aclanthology.org/2021.mrl-1.11/
[336/1902] Processing: https://aclanthology.org/2021.law-1.7/
[337/1902] Processing: https://aclanthology.org/2021.iwslt-1.21/
[338/1902] Processing: https://aclanthology.org/2021.iwslt-1.22/
[339/1902] Processing: https://aclanthology.org/2021.iwpt-1.9/
[340/1902] Processing: https://aclanthology.org/2021.hcinlp-1.12/
[341/1902] Processing: https://aclanthology.org/2021.gwc-1.3/
[342/1902] Processing: https://aclanthology.org/2021.findings-emnlp.4/
[343/1902] Processing: https://aclanthology.org/2021.findings-emnlp.281/
[344/1902] Processing: https://aclanthology.org/2021.findings-emnlp.282/
[345/1902] Processing: https://aclanthology.org/2021.findings-emnlp.382/
[346/1902] Processing: https://aclanthology.org/2021.findings-emnlp.410/
[347/1902] Processing: https://aclanthology.org/2021.emnlp-main.99/
[348/1902] Processing: https://aclanthology.org/2

[380/1902] Processing: https://aclanthology.org/2020.sltu-1.42/
[381/1902] Processing: https://aclanthology.org/2020.signlang-1.11/
[382/1902] Processing: https://aclanthology.org/2020.sigmorphon-1.1/
[383/1902] Processing: https://aclanthology.org/2020.semeval-1.275/
[384/1902] Processing: https://aclanthology.org/2020.rail-1.3/
[385/1902] Processing: https://aclanthology.org/2020.rail-1.4/
[386/1902] Processing: https://aclanthology.org/2020.rail-1.5/
[387/1902] Processing: https://aclanthology.org/2020.rail-1.7/


[388/1902] Processing: https://aclanthology.org/2020.rail-1.8/


[389/1902] Processing: https://aclanthology.org/2020.rail-1.9/
[390/1902] Processing: https://aclanthology.org/2020.nlpcovid19-acl.16/
[391/1902] Processing: https://aclanthology.org/2020.nlpcovid19-2.5/
[392/1902] Processing: https://aclanthology.org/2020.lrec-1.151/
[393/1902] Processing: https://aclanthology.org/2020.lrec-1.258/
[394/1902] Processing: https://aclanthology.org/2020.lrec-1.301/


...

[1533/1902] Processing: https://aclanthology.org/2023.cl-2.2/
[1534/1902] Processing: https://aclanthology.org/2023.cawl-1.4/
[1535/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.11/
[1536/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.12/
[1537/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.15/
[1538/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.19/
[1539/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.27/
[1540/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.30/
[1541/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.38/
[1542/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.47/
[1543/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.53/
[1544/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.58/
[1545/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.66/
[1546/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.71/
[1547/1902] Processing: https://ac

[1611/1902] Processing: https://aclanthology.org/2020.wanlp-1.9/
[1612/1902] Processing: https://aclanthology.org/2020.wanlp-1.17/
[1613/1902] Processing: https://aclanthology.org/2020.wanlp-1.19/
[1614/1902] Processing: https://aclanthology.org/2020.wanlp-1.20/
[1615/1902] Processing: https://aclanthology.org/2020.wanlp-1.21/
[1616/1902] Processing: https://aclanthology.org/2020.semeval-1.200/
[1617/1902] Processing: https://aclanthology.org/2020.semeval-1.253/
[1618/1902] Processing: https://aclanthology.org/2020.semeval-1.296/
[1619/1902] Processing: https://aclanthology.org/2020.rdsm-1.7/
[1620/1902] Processing: https://aclanthology.org/2020.osact-1.1/
[1621/1902] Processing: https://aclanthology.org/2020.osact-1.14/
[1622/1902] Processing: https://aclanthology.org/2020.osact-1.15/
[1623/1902] Processing: https://aclanthology.org/2020.nlptea-1.2/
[1624/1902] Processing: https://aclanthology.org/2020.lrec-1.174/


[1625/1902] Processing: https://aclanthology.org/2020.lrec-1.223/
[1626/1902] Processing: https://aclanthology.org/2020.lrec-1.308/
[1627/1902] Processing: https://aclanthology.org/2020.lrec-1.473/
[1628/1902] Processing: https://aclanthology.org/2020.lrec-1.480/
[1629/1902] Processing: https://aclanthology.org/2020.lrec-1.506/
[1630/1902] Processing: https://aclanthology.org/2020.lrec-1.508/
[1631/1902] Processing: https://aclanthology.org/2020.lrec-1.531/
[1632/1902] Processing: https://aclanthology.org/2020.lrec-1.657/
[1633/1902] Processing: https://aclanthology.org/2020.lrec-1.761/
[1634/1902] Processing: https://aclanthology.org/2020.lrec-1.768/
[1635/1902] Processing: https://aclanthology.org/2020.lrec-1.816/
[1636/1902] Processing: https://aclanthology.org/2020.lrec-1.817/
[1637/1902] Processing: https://aclanthology.org/2020.latechclfl-1.3/
[1638/1902] Processing: https://aclanthology.org/2020.fever-1.2/
[1639/1902] Processing: https://aclanthology.org/2020.emnlp-main.455/
[16

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
# === Step 1: Upload the Excel file ===
from google.colab import files
uploaded = files.upload()

# === Step 2: Import dependencies ===
import pandas as pd
import requests
from io import BytesIO
from PyPDF2 import PdfReader
import re
import time
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
from urllib.parse import urlparse

# === Step 3: Read file ===
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)
df.columns = df.columns.str.lower()

if 'url' not in df.columns:
    raise ValueError("The Excel file must contain a column named 'url'.")

urls = df['url'].dropna().tolist()

# === Step 4: Utility functions ===
def clean_text(text):
    return ILLEGAL_CHARACTERS_RE.sub('', text.strip())

def extract_affiliations_from_text(text):
    lines = text.split("\n")
    affiliations = set()
    for line in lines:
        line = clean_text(line)
        if len(line) < 5 or '@' in line:
            continue
        if any(keyword in line.lower() for keyword in [
            'university', 'institute', 'college', 'school', 'laboratory', 'lab',
            'dept.', 'department', 'center', 'centre', 'academy'
        ]):
            cleaned = re.sub(r'\b[\w.-]+?@\w+?\.\w+?\b', '', line)
            cleaned = re.sub(r'\s+', ' ', cleaned.strip())
            cleaned = ILLEGAL_CHARACTERS_RE.sub('', cleaned)
            if 5 < len(cleaned) < 200:
                affiliations.add(cleaned)
    return sorted(affiliations)

def extract_acl_pdf_url(url):
    if url.endswith('/'):
        url = url[:-1]
    paper_id = url.split('/')[-1]
    return f"https://aclanthology.org/{paper_id}.pdf"

def extract_acl_affiliations(url):
    try:
        pdf_url = extract_acl_pdf_url(url)
        response = requests.get(pdf_url, timeout=30)
        response.raise_for_status()
        reader = PdfReader(BytesIO(response.content))
        first_page = reader.pages[0].extract_text()
        if not first_page or first_page.strip() == "":
            return "No text extracted from first page"
        affiliations = extract_affiliations_from_text(first_page)
        return "; ".join(affiliations) if affiliations else "No affiliations matched"
    except Exception as e:
        return f"ERROR: {str(e)}"

def extract_s2_paper_id(url):
    return url.strip('/').split('/')[-1]

def extract_semantic_metadata(paper_id):
    api_url = f"https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=title,authors"
    try:
        res = requests.get(api_url, timeout=15)
        data = res.json()
        title = data.get("title", "N/A")
        authors = data.get("authors", [])
        institutions = set()
        for author in authors:
            affs = author.get("affiliations", [])
            if isinstance(affs, list):
                institutions.update([clean_text(a) for a in affs if isinstance(a, str)])
        return title, "; ".join(institutions) if institutions else "No affiliations listed"
    except Exception as e:
        return "ERROR", f"Failed: {str(e)}"

# === Step 5: Process all URLs ===
results = []
for idx, url in enumerate(urls, 1):
    print(f"[{idx}/{len(urls)}] Processing: {url}")
    if 'aclanthology.org' in url:
        affiliation = extract_acl_affiliations(url)
        results.append({"URL": url, "Title": "N/A", "Affiliations": affiliation})
    elif 'semanticscholar.org' in url:
        paper_id = extract_s2_paper_id(url)
        title, aff = extract_semantic_metadata(paper_id)
        results.append({"URL": url, "Title": title, "Affiliations": aff})
    else:
        results.append({"URL": url, "Title": "Unknown", "Affiliations": "Unsupported URL"})

# === Step 6: Save and download result ===
result_df = pd.DataFrame(results)
output_file = "institution_extraction_result.xlsx"
result_df.to_excel(output_file, index=False)
files.download(output_file)


Saving institution.xlsx to institution (2).xlsx
[1/1902] Processing: https://aclanthology.org/2025.wacl-1.1/
[2/1902] Processing: https://aclanthology.org/2025.wacl-1.9/
[3/1902] Processing: https://aclanthology.org/2025.wacl-1.10/
[4/1902] Processing: https://aclanthology.org/2025.resourceful-1.34/
[5/1902] Processing: https://aclanthology.org/2025.nlp4dh-1.26/
[6/1902] Processing: https://aclanthology.org/2025.naacl-long.92/
[7/1902] Processing: https://aclanthology.org/2025.naacl-long.139/
[8/1902] Processing: https://aclanthology.org/2025.naacl-long.338/
[9/1902] Processing: https://aclanthology.org/2025.naacl-long.426/
[10/1902] Processing: https://aclanthology.org/2025.naacl-long.602/
[11/1902] Processing: https://aclanthology.org/2025.naacl-short.2/
[12/1902] Processing: https://aclanthology.org/2025.naacl-short.35/
[13/1902] Processing: https://aclanthology.org/2025.loresmt-1.4/
[14/1902] Processing: https://aclanthology.org/2025.loresmt-1.6/
[15/1902] Processing: https://aclan

[334/1902] Processing: https://aclanthology.org/2021.mtsummit-research.6/
[335/1902] Processing: https://aclanthology.org/2021.mrl-1.11/
[336/1902] Processing: https://aclanthology.org/2021.law-1.7/
[337/1902] Processing: https://aclanthology.org/2021.iwslt-1.21/
[338/1902] Processing: https://aclanthology.org/2021.iwslt-1.22/
[339/1902] Processing: https://aclanthology.org/2021.iwpt-1.9/
[340/1902] Processing: https://aclanthology.org/2021.hcinlp-1.12/
[341/1902] Processing: https://aclanthology.org/2021.gwc-1.3/
[342/1902] Processing: https://aclanthology.org/2021.findings-emnlp.4/
[343/1902] Processing: https://aclanthology.org/2021.findings-emnlp.281/
[344/1902] Processing: https://aclanthology.org/2021.findings-emnlp.282/
[345/1902] Processing: https://aclanthology.org/2021.findings-emnlp.382/
[346/1902] Processing: https://aclanthology.org/2021.findings-emnlp.410/
[347/1902] Processing: https://aclanthology.org/2021.emnlp-main.99/
[348/1902] Processing: https://aclanthology.org/2

[380/1902] Processing: https://aclanthology.org/2020.sltu-1.42/
[381/1902] Processing: https://aclanthology.org/2020.signlang-1.11/
[382/1902] Processing: https://aclanthology.org/2020.sigmorphon-1.1/
[383/1902] Processing: https://aclanthology.org/2020.semeval-1.275/
[384/1902] Processing: https://aclanthology.org/2020.rail-1.3/
[385/1902] Processing: https://aclanthology.org/2020.rail-1.4/
[386/1902] Processing: https://aclanthology.org/2020.rail-1.5/
[387/1902] Processing: https://aclanthology.org/2020.rail-1.7/


[388/1902] Processing: https://aclanthology.org/2020.rail-1.8/


[389/1902] Processing: https://aclanthology.org/2020.rail-1.9/
[390/1902] Processing: https://aclanthology.org/2020.nlpcovid19-acl.16/
[391/1902] Processing: https://aclanthology.org/2020.nlpcovid19-2.5/
[392/1902] Processing: https://aclanthology.org/2020.lrec-1.151/
[393/1902] Processing: https://aclanthology.org/2020.lrec-1.258/
[394/1902] Processing: https://aclanthology.org/2020.lrec-1.301/


...

[1533/1902] Processing: https://aclanthology.org/2023.cl-2.2/
[1534/1902] Processing: https://aclanthology.org/2023.cawl-1.4/
[1535/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.11/
[1536/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.12/
[1537/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.15/
[1538/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.19/
[1539/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.27/
[1540/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.30/
[1541/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.38/
[1542/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.47/
[1543/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.53/
[1544/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.58/
[1545/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.66/
[1546/1902] Processing: https://aclanthology.org/2023.arabicnlp-1.71/
[1547/1902] Processing: https://ac

[1611/1902] Processing: https://aclanthology.org/2020.wanlp-1.9/
[1612/1902] Processing: https://aclanthology.org/2020.wanlp-1.17/
[1613/1902] Processing: https://aclanthology.org/2020.wanlp-1.19/
[1614/1902] Processing: https://aclanthology.org/2020.wanlp-1.20/
[1615/1902] Processing: https://aclanthology.org/2020.wanlp-1.21/
[1616/1902] Processing: https://aclanthology.org/2020.semeval-1.200/
[1617/1902] Processing: https://aclanthology.org/2020.semeval-1.253/
[1618/1902] Processing: https://aclanthology.org/2020.semeval-1.296/
[1619/1902] Processing: https://aclanthology.org/2020.rdsm-1.7/
[1620/1902] Processing: https://aclanthology.org/2020.osact-1.1/
[1621/1902] Processing: https://aclanthology.org/2020.osact-1.14/
[1622/1902] Processing: https://aclanthology.org/2020.osact-1.15/
[1623/1902] Processing: https://aclanthology.org/2020.nlptea-1.2/
[1624/1902] Processing: https://aclanthology.org/2020.lrec-1.174/


[1625/1902] Processing: https://aclanthology.org/2020.lrec-1.223/
[1626/1902] Processing: https://aclanthology.org/2020.lrec-1.308/
[1627/1902] Processing: https://aclanthology.org/2020.lrec-1.473/
[1628/1902] Processing: https://aclanthology.org/2020.lrec-1.480/
[1629/1902] Processing: https://aclanthology.org/2020.lrec-1.506/
[1630/1902] Processing: https://aclanthology.org/2020.lrec-1.508/
[1631/1902] Processing: https://aclanthology.org/2020.lrec-1.531/
[1632/1902] Processing: https://aclanthology.org/2020.lrec-1.657/
[1633/1902] Processing: https://aclanthology.org/2020.lrec-1.761/
[1634/1902] Processing: https://aclanthology.org/2020.lrec-1.768/
[1635/1902] Processing: https://aclanthology.org/2020.lrec-1.816/
[1636/1902] Processing: https://aclanthology.org/2020.lrec-1.817/
[1637/1902] Processing: https://aclanthology.org/2020.latechclfl-1.3/
[1638/1902] Processing: https://aclanthology.org/2020.fever-1.2/
[1639/1902] Processing: https://aclanthology.org/2020.emnlp-main.455/
[16

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>